In [1]:
import itertools
from hs_brain_mlops.dataset_management.dataset_manager_factory import get_dataset_manager
from hs_gimme.db_facade.elastic_search.elastic_search_client import get_elastic_search_client
from hs_fetch.workflows.factories.past_candidates_workflows_factory import PastCandidatesWorkflowsFactory
from collections import defaultdict, Counter

/Users/dima/.virtualenvs/hs/lib/python3.9/site-packages/xgboost/compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/Users/dima/.virtualenvs/hs/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


2024-01-15T12:35:20.090852Z [info     ] Read abbreviation sets         environment=local file_path=/Users/dima/hiredscore/grading/algorithms/hs_algorithms/abbreviations/abbrevations_provider.py function_name=_get_abbreavations_from_file hostname=Dima-Shulga-MacBook-Pro line_number=34 module=hs_algorithms.abbreviations.abbrevations_provider num_terms__numeric=91 pid=63533
2024-01-15T12:35:20.105529Z [info     ] Explainable job type: loaded aliases to job types environment=local file_path=/Users/dima/hiredscore/grading/algorithms/hs_algorithms/explainable_job_type/explainable_job_type.py function_name=load_job_type_data hostname=Dima-Shulga-MacBook-Pro line_number=109 module=hs_algorithms.explainable_job_type.explainable_job_type num_aliases__numeric=365 num_job_types__numeric=214 pid=63533


/Users/dima/.virtualenvs/hs/lib/python3.9/site-packages/pydantic/_internal/_config.py:317: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
  warnings.warn(message, UserWarning)


In [9]:
account = 'tokyo'
tag = 'baseline_jan_12_2'

In [3]:
dataset_manager = get_dataset_manager()
dataset = list(dataset_manager.load('fetch_evaluation_' + account))

2024-01-15T12:35:20.492297Z [info     ] Running DVC command ['DVC_IGNORE_ISATTY=1', 'dvc', 'pull', '--force', PosixPath('/Users/dima/hiredscore/brain_mlops/hs_brain_mlops/dataset_management/datasets/fetch_evaluation_tokyo/data')] environment=local file_path=/Users/dima/hiredscore/brain_mlops/hs_brain_mlops/dataset_management/dvc_manager/dvc_facade.py function_name=_run hostname=Dima-Shulga-MacBook-Pro line_number=31 module=hs_brain_mlops.dataset_management.dvc_manager.dvc_facade pid=63533


Fetching
!
  0% |          |0/? [00:00<?,    ?files/s]
                                           
!
  0% Querying remote cache|          |0/1 [00:00<?,    ?files/s]
100% Querying remote cache|██████████|1/1 [00:00<00:00,  5.85files/s]
                                                                     
!
  0%|          |Fetching from s3                      0/3 [00:00<?,     ?file/s]
  0%|          |Fetching from s3                      0/2 [00:00<?,     ?file/s]

!

  0%|          |hs-brain-datasets/dvc/files/md5/0.00/313k [00:00<?,        ?B/s]

 16%|█▌        |hs-brain-datasets/dvc/files50.5k/313k [00:00<00:00,     349kB/s]

 38%|███▊      |hs-brain-datasets/dvc/files/118k/313k [00:00<00:00,     430kB/s]

 64%|██████▎   |hs-brain-datasets/dvc/files/199k/313k [00:00<00:00,     492kB/s]

 92%|█████████▏|hs-brain-datasets/dvc/files/288k/313k [00:00<00:00,     553kB/s]

                                                                                
 50%|█████     |Fetching from s3   

M       fetch_evaluation_tokyo/data/
1 file modified and 3 files fetched


In [ ]:
dataset[0]['talents'][0]['e']

In [ ]:
for x in dataset[0]['talents'][0]['talent_sources']:
    print(x['source'])

In [5]:
brain_es = get_elastic_search_client('production_brain', tries=4, application='fast_fetch')

Found credentials in shared credentials file: ~/.aws/credentials


In [ ]:
mongo = gmcdb('production_brain', account)

In [12]:
for index_name, data in brain_es.indices.get_alias('*').items():
    if account in index_name and tag in index_name:
        print(index_name)
        break

tokyo_fast_fetch_production_brain_baseline_jan_12_2_active


In [7]:
cands_dict = defaultdict(list)
for req_and_talents in dataset:
    for talent in req_and_talents['talents']:
        cands_dict[talent['evaluation_group']].append(talent['current_talent_id'])

In [ ]:
for cands_group, cands in cands_dict.items():
    print(cands_group, len(mongo['application_' + tag].distinct('current_talent_id', {'current_talent_id': {'$in': cands}})), '/', len(cands))

In [ ]:
original_req_ids = set()
cands_data = {}
for req_and_talents in dataset:
    req = req_and_talents['req']
    talents = req_and_talents['talents']
    for talent in req_and_talents['talents']:
        if talent['original_req_id']:
            talent_id = talent['current_talent_id']
            req_id = talent['original_req_id']
            original_req_ids.add(req_id)
            cands_data[talent_id] = talent
            cands_data[talent_id]['req_id'] = req_id
original_req_ids  = list(set(original_req_ids))
len(original_req_ids)

In [ ]:
for req_and_talents in dataset:
    req = req_and_talents['req']
    talents = req_and_talents['talents']
    print(req['_id'], len(talents))

In [ ]:
mongo['req_' + tag].count_documents({'_id': {'$in': original_req_ids}}), '/', len(original_req_ids)

In [ ]:
all_req_ids = list(set([r['req']['_id'] for r in dataset]))
len(all_req_ids)

In [ ]:
print(mongo['req_' + tag].count_documents({'_id': {'$in': all_req_ids}}), '/', len(all_req_ids))

In [14]:
missing = defaultdict(set)
for cands_group, cands in cands_dict.items():
    q = {
        '_source': ['current_talent_id'],
        'query': {'bool': {'filter':
                               [
                                   {"bool":
                                       {"filter":
                                           {"terms": {
                                               'current_talent_id': cands}}
                                       }
                                   }
                               ]
        }
        }
    }
    res = brain_es.search(index=index_name, body=q, size=5000)
    talent_ids = [h['_source']['current_talent_id'] for h in res['hits']['hits']]
    print(cands_group, len(set(talent_ids)), '/', len(set(cands)))
    
    missing[cands_group] = set(cands) - set(talent_ids)

rejected 159 / 159
offer 159 / 159
random 146 / 146
random_out_category 144 / 144
random_in_category 140 / 140


In [ ]:
talent_id = list(missing['offer'])[0]

In [ ]:
apps = list(mongo['application_' + tag].find({'current_talent_id': talent_id}))
len(app)

In [ ]:
list(missing['random'])[0]

In [ ]:
req_id = app['req_id']
req_id

In [ ]:
req = mongo['req_' + tag].find_one(req_id)
bool(req)

In [ ]:
dataset[0]['talents'][0].keys()

In [ ]:
dataset[0]['talents'][0]['talent'].keys()

In [ ]:
workflow_factory = PastCandidatesWorkflowsFactory(LoggingService(), 'production_brain', 'moscow')

In [ ]:
account_config = get_fetch_config('moscow', country=None)

In [ ]:
account_config.get_max_days_since_apply_for_load()

In [ ]:
mongo = gmcdb('production_brain', 'moscow')

In [ ]:
prod_mongo = gmcdb('production', 'moscow')

In [ ]:
apps = list(prod_mongo.application.find({'current_talent_id': {'$in': cands}}, ['date_applied']))

In [ ]:
from collections import Counter

In [ ]:
Counter(a['date_applied'].year for a in apps).most_common()

In [ ]:
from hs_tree_blenders_creation.batch_runners.batch_runs_data_layer import get_fetch_data_layer
from hs_fetch.data_access.potential_candidate_loading.loader_strategies import DataLayerLoaderStrategy

In [ ]:
dl = get_fetch_data_layer(
        'moscow',
        environment='production_brain',
        req_input_tag='cosine_2',
        talent_input_tag='cosine_2',
    )

In [ ]:
indexing_workflow = workflow_factory.create_index_candidates_for_fast_fetch_workflow(
                account_config.get_max_days_since_apply_for_load(), 
                datetime.today(), '',
                logging_service=LoggingService(), 
                loader_strategy=DataLayerLoaderStrategy(dl), es_tag='test')

In [ ]:
res = indexing_workflow.get_candidate_data(list(missing['rejected'])[-50:], req=None)
sum([bool(r) for r in res])

In [ ]:
none_cands = []
for c, r in zip(cands[-20:], res):
    if not r:
        none_cands.append(c)
none_cands       

In [ ]:
res = indexing_workflow.get_candidate_data(['b617460d-79b1-403b-8dd3-1ab14b3d0735',
 '0540c11b-7720-4c6c-8fb8-241dbb4f8851',
 '73b96d80-b07a-4b82-8e16-ec5e30bc5663'], req=None)
sum([bool(r) for r in res])

In [ ]:
class _FetchEvaluationDataset:

    FETCH_EVALUATION_DATASET = 'fetch_evaluation'

    def __init__(self, version, test_size):
        self._version = version
        self._test_size = test_size
        self._dataset_manager = get_dataset_manager()

    def _get_fetch_test_set(self):
        return self._dataset_manager.load(self.FETCH_EVALUATION_DATASET, self._version)

    def get_test_set_for_account(self, account_id):
        all_accounts_test_set = self._get_fetch_test_set()
        account_test_set = all_accounts_test_set.get(account_id, {})
        return get_dict_sample(account_test_set, self._test_size), account_test_set

    def get_version(self):
        return self._dataset_manager.get_translated_version(self.FETCH_EVALUATION_DATASET, self._version)

In [ ]:
ds = _FetchEvaluationDataset('LATEST', 2000)

In [ ]:
def get_dict_sample(d, size):
    if size is None or len(d) <= size:
        return d
    keys = get_sample(list(d), size)
    values = [d[k] for k in keys]
    return dict(zip(keys, values))

In [ ]:
x = ds.get_test_set_for_account('moscow')

In [ ]:
len(x[0])

In [ ]:
len(x[1])

In [ ]:
type(x[0])

In [ ]:
req_ids = list(x[0].keys())

In [ ]:
len(req_ids)

In [ ]:
mongo

In [ ]:
mongo_brain = gmcdb('production_brain', 'moscow')

In [ ]:
reqs = list(mongo_brain.req_healthcare_3.find())

In [ ]:
Counter(r['activation_data']['is_country_supported'] for r in reqs)

In [ ]:
dataset.keys()

In [ ]:
cand_to_group = {}
cand_to_req = {}
for req, req_cands in dataset['moscow'].items():
    for cand, cand_data in req_cands.items():
        cand_to_group[cand] = cand_data['evaluation_group']
        cand_to_req[cand] = req

In [ ]:
for c in ['b617460d-79b1-403b-8dd3-1ab14b3d0735',
 '0540c11b-7720-4c6c-8fb8-241dbb4f8851',
 '73b96d80-b07a-4b82-8e16-ec5e30bc5663']:
    print(c, cand_to_group[c], cand_to_req[c])

In [ ]:
from hs_tree_blenders_creation.batch_runners.data_layers.fetch_data_layer import FetchMongoDataLayer

In [ ]:
dl = FetchMongoDataLayer('moscow', 'production_brain')

In [ ]:
x = list(dl.get_talents_by_source('application', ['b617460d-79b1-403b-8dd3-1ab14b3d0735',
 '0540c11b-7720-4c6c-8fb8-241dbb4f8851',
 '73b96d80-b07a-4b82-8e16-ec5e30bc5663']))

In [ ]:
len(x)

In [ ]:
set(t['current_talent_id'] for t in x)

In [ ]:
len(dataset['moscow']['MCY895480'])

In [ ]:
dataset['moscow']['MCY895480']

In [ ]:
q = {
    '_source': ['current_talent_id'],
    'query': {'bool': {'filter':
                           [
                               {"bool":
                                   {"filter":
                                       {"terms": {
                                           'current_talent_id': ['b617460d-79b1-403b-8dd3-1ab14b3d0735',
 '0540c11b-7720-4c6c-8fb8-241dbb4f8851',
 '73b96d80-b07a-4b82-8e16-ec5e30bc5663']}}
                                   }
                               }
                           ]
    }
    }
}

res = brain_es.search(index='moscow_fast_fetch_production_brain_healthcare_5_active', body=q, size=5000)
talent_ids = [h['_source']['current_talent_id'] for h in res['hits']['hits']]
len(talent_ids)

In [ ]:

len(apps)

In [ ]:
from collections import Counter, defaultdict

In [ ]:
cnt = Counter()
apps = []
reqs = []
for account in dataset.keys():
    mongo = gmcdb('production_brain', account)
    apps += list(mongo.application_healthcare_5.find({}, ['req_id', 'current_talent_id']))
    reqs += list(mongo.req_healthcare_5.find({}, ['activation_data']))
    
print(len(apps))
for a in apps:
    cnt[a['current_talent_id']] += 1
cnt.most_common(5)

In [ ]:
len(reqs), reqs[0]

In [ ]:
cnts = defaultdict(Counter)
for r in reqs:
    for k, v in r['activation_data'].items():
        cnts[k][v] += 1

In [ ]:
for k, cnt in cnts.items():
    print(k)
    print(cnt.most_common(5))
    print()

In [ ]:
brain_reqs = mongo.req_cosine_2.distinct('_id', {'_id': {'$in': reqs}})

In [ ]:
prod_mongo.req.count_documents({'_id': {'$in': reqs}})

In [ ]:
prod_reqs = prod_mongo.req.distinct('_id', {'_id': {'$in': reqs}})

In [ ]:
len(brain_reqs)

In [ ]:
len(prod_reqs)

In [ ]:
missing_reqs = list(set(prod_reqs) - set(brain_reqs))
len(missing_reqs)

In [ ]:
len(prod_mongo.application.distinct('current_talent_id', {'req_id': {'$in': missing_reqs}}))

In [ ]:
len(mongo.application_cosine_2.distinct('current_talent_id'))

In [ ]:
mongo.application_cosine_2.find_one({'current_talent_id': '06d978f6-ed46-4dc4-b575-2b086506f9d9'})

In [ ]:
'b617460d-79b1-403b-8dd3-1ab14b3d0735' in cands

In [ ]:
in_mongo = set(mongo.application_cosine_2.distinct('current_talent_id'))
len(in_mongo)

In [ ]:
in_mongo.issubset(set(cands))

In [ ]:
list(in_mongo - set(cands))[:10]

In [ ]:
len(set(cands) - in_mongo)

In [ ]:
'67aa433a-2126-401b-a328-047703ffdd94' in cands

In [ ]:
mongo.req_cosine_2.count_documents({'_id': {'$in': ['MCY810112',
 'MCY816066',
 'MCY803076',
 'MCY831880',
 'MCY835696',
 'MCY794664',
 'MCY824999',
 'MCY829242',
 'MCY798899',
 'MCY813109',
 'MCY835204',
 'MCY798803',
 'MCY831260',
 'MCY837050',
 'MCY817412',
 'MCY837329',
 'MCY795810',
 'MCY798265',
 'MCY799975',
 'MCY795908',
 'MCY829146',
 'MCY800432',
 'MCY835313',
 'MCY816373',
 'MCY821455',
 'MCY807867',
 'MCY796256',
 'MCY832782',
 'MCY835742',
 'MCY835103',
 'MCY830709',
 'MCY809630',
 'MCY850318',
 'MCY834483',
 'MCY843129',
 'MCY826616',
 'MCY798612',
 'MCY806073',
 'MCY795166',
 'MCY821039',
 'MCY804847',
 'MCY816521',
 'MCY835775',
 'MCY798206',
 'MCY800475',
 'MCY799269',
 'MCY834119',
 'MCY831423',
 'MCY804846',
 'MCY798532',
 'MCY798732']}})

In [ ]:
from hs_fetch.fast_fetch.fast_fetch_indexing import FastFetchIndexing

In [ ]:
indexing = FastFetchIndexing('moscow', 'production', None)

In [ ]:
res = indexing.get_cand

In [ ]:
len(missing['offer'])

In [ ]:
cands_data['0898f140-8343-41a7-adb1-bbd267bbb995']

In [ ]:
dataset['athens']['ALL117991']

In [ ]:
'ALL117991' in original_req_ids

In [ ]:
for r, r_data in dataset['athens'].items():
    for c, c_data in r_data.items():
        if c_data['original_req_id'] == 'ALL117991':
            print(r, c, c_data)
            print()

In [ ]:
d = {
    1: 'one',
    

In [ ]:
{
    'req': xxx,
    'talents': 
    [
        {
            'current_talent_id': xxx,
            'evaluation_group': xxx,
            'applications': [
                { 
                    'app': app,
                    'original_req' original_req
                },
            ],
            'crm_candidates': [],
            'empoyees': [],         
        }   
    ]
}

In [ ]:
cnt = Counter()
for req_and_talents in dataset:
    for t in req_and_talents['talents']:
        cnt[len(t['talent_sources'])] += 1

In [ ]:
cnt.most_common()

In [ ]:
for req_and_talents in dataset:
    req = req_and_talents['req']
    if req['_id'] == 'TRN2003814':
        print('found')
        break

In [ ]:
talents = req_and_talents['talents']

In [ ]:
for talent in talents:
    if talent['current_talent_id'] == 'b522ff92-5eb5-4f60-9edf-2f19298fd9b1':
        print('found')
        break

In [ ]:
talent.keys()

In [ ]:
talent['talent_sources'][0]